In [1]:
import pandas as pd
import os
import seaborn as sns
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv(f'{os.getcwd()}/data_after_eda.csv')
del df['Unnamed: 0']

In [4]:
df.head()

,title,average_rating,director,writer,metascore,cast,country_of_origin,languages,runtime,genre,budget,worldwide_gross,release_year,profit_or_loss,p&l_rate,lang_cnt,country_cnt,coproduction
0,The Shawshank Redemption,9.3,Frank Darabont,"Stephen King, Frank Darabont",82,"Morgan Freeman, Bob Gunton, William Sadler, C...",United States,English,142,Drama,25000000,29332133,1994,4332133,17.33,1,1,0
1,The Godfather,9.2,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola",100,"Al Pacino, James Caan, Diane Keaton, Richard ...",United States,"English, Italian, Latin",175,"Crime, Drama",6000000,250342198,1978,244342198,4072.37,3,1,0
2,The Dark Knight,9.0,Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S. Goyer",84,"Heath Ledger, Aaron Eckhart, Michael Caine, M...","United States, United Kingdom","English, Mandarin",152,"Action, Crime, Drama",185000000,1009053678,2008,824053678,445.43,2,2,1
3,Schindler's List,9.0,Steven Spielberg,"Thomas Keneally, Steven Zaillian",95,"Ralph Fiennes, Ben Kingsley, Caroline Goodall...",United States,"English, Hebrew, German, Polish, Latin",195,"Biography, Drama, History",22000000,322161245,1993,300161245,1364.37,5,1,0
4,12 Angry Men,9.0,Sidney Lumet,Reginald Rose,97,"Lee J. Cobb, Martin Balsam, John Fiedler, E.G...",United States,English,96,"Crime, Drama",350000,2945,1957,-347055,-99.16,1,1,0


## Проверка гипотез

### Гипотеза 1: Фильмы с определённым жанром (например, драмы) чаще получают высокие оценки

Данная гипотеза выдвинута на основании предположения о том, что фильмы определённого жанра могут обладать характерными особенностями, которые делают их более привлекательными для зрителей и критиков.

Для проведения статистического теста можно разбить фильмы на жанры и сравнить их средние рейтинги:

In [9]:
drama_movies = df[df['genre'].str.contains('Drama', case=False, na=False)][['average_rating', 'genre']]
other_movies = df[~df['genre'].str.contains('Drama', case=False, na=False)][['average_rating', 'genre']]

print(f'Среднее значение рейтинга у фильмов с жанром Drama: {drama_movies.average_rating.mean()}')
print(f'Среднее значение рейтинга у фильмов без жанра Drama: {other_movies.average_rating.mean()}')
print()

print('Проверка нормальности данных в выборках по рейтингу')
print(ss.shapiro(drama_movies.average_rating))  # для драм
print(ss.shapiro(other_movies.average_rating))  # для остальных жанров
print()

print('Проверка на равенство дисперсий')
print(ss.levene(drama_movies.average_rating, other_movies.average_rating))

Среднее значение рейтинга у фильмов с жанром Drama: 7.612086874409821
Среднее значение рейтинга у фильмов без жанра Drama: 7.603535353535355

Проверка нормальности данных в выборках по рейтингу
ShapiroResult(statistic=np.float64(0.9005525405782255), pvalue=np.float64(1.2056483539575647e-25))
ShapiroResult(statistic=np.float64(0.9093519031110463), pvalue=np.float64(1.1742995008129563e-14))

Проверка на равенство дисперсий
LeveneResult(statistic=np.float64(0.8522876717058208), pvalue=np.float64(0.3560592469154229))


Полученные результаты тестов свидетельствуют о том, что данные распределены ненормально, а дисперсии в выборках можно считать равными. Вследствие этого использование классического t-теста для определения уровня статистической значимости разниц становится  некорректным: вместо указанного теста необходимо использовать непараметрический тест Манна-Уитни:

In [12]:
ss.mannwhitneyu(drama_movies.average_rating, other_movies.average_rating)

MannwhitneyuResult(statistic=np.float64(209926.0), pvalue=np.float64(0.9726116038158621))

**Вывод : Результаты теста Манна-Уитни показали отсутствие статистически значимой разницы между выборками. Таким образом, гипотеза о том, что фильмы с жанром Drama имеют более высокие рейтинги, отвергается — статистически значимых различий в оценках между фильмами с жанром Drama и без него не выявлено.**

### Гипотеза 2: Фильмы с совпадением режиссёра и сценариста более успешны

Данная гипотеза предполагает, что если режиссёр и сценарист фильма - один и тот же человек, то, возможно, такой фильм имеет более четкое видение и концепцию, что приводит к более высоким результатам в рейтингах и в кассовых сборах.

Для проверки гипотезы можно разбить фильмы на две группы: с совпадением режиссёра и сценариста и без. Затем можно проверить средний рейтинг или сборы в этих группах:

In [ ]:
same_director_writer = df.query('director == writer')[['average_rating', 'worldwide_gross']]
diff_director_writer = df.query('director != writer')[['average_rating', 'worldwide_gross']]

print(f'Средний рейтинг у фильмов с одинаковым режиссёром и сценаристом: {same_director_writer.average_rating.mean()}')
print(f'Средний рейтинг у фильмов с разными режиссёром и сценаристом: {diff_director_writer.average_rating.mean()}')


Средний рейтинг у фильмов с одинаковым режиссёром и сценаристом: 7.573214285714285
Средний рейтинг у фильмов с разными режиссёром и сценаристом: 7.616409423233144
Проверка на значимость разницы с t-тестом
TtestResult(statistic=np.float64(-1.6641201978387392), pvalue=np.float64(0.09630403284771445), df=np.float64(1453.0))
Проверка с помощью теста Манна-Уитни
MannwhitneyuResult(statistic=np.float64(131428.5), pvalue=np.float64(0.26257580849540907))


Полученные результаты тестов свидетельствуют о том, что данные распределены ненормально, а дисперсии в рассматриваемых выборках неравны. Вследствие этого использование классического t-теста для определения уровня статистической значимости разниц становится некорректным: вместо указанного теста необходимо использовать непараметрический тест Манна-Уитни

In [13]:
ss.mannwhitneyu(same_director_writer.average_rating, diff_director_writer.average_rating)

MannwhitneyuResult(statistic=np.float64(131428.5), pvalue=np.float64(0.26257580849540907))

**Вывод : Результаты непараметрического теста Манна-Уитни не позволяют отвергнуть нулевую гипотезу о равенстве распределений. Таким образом, гипотеза о том, что фильмы с одним и тем же автором сценария и режиссёром получают более высокие оценки, отвергается — статистически значимой разницы в рейтингах между этими группами фильмов не выявлено.**

### Гипотеза 3: Фильмы с более высоким бюджетом имеют большую прибыль

Данная гипотеза основана на предположении, что крупнобюджетные фильмы чаще обладают высококачественными технологиями съемки и привлечением известных актёров, что увеличивает их привлекательность для широкой аудитории и, как следствие, приносит большую прибыль.

Для анализа разделим фильмы на две группы по уровню бюджета — ниже и выше медианного значения. Далее сравним среднее значение прибыли в этих группах и проверим выборки на нормальность и равенство дисперсий:

In [15]:
median_budget = df['budget'].median()
g1 = df[df['budget'] <= median_budget][['profit_or_loss', 'budget']]  # низкий бюджет
g2 = df[df['budget'] > median_budget][['profit_or_loss', 'budget']]   # высокий бюджет

print(f'Средняя прибыль у фильмов с низким бюджетом: {round(g1.profit_or_loss.mean(), 2)}')
print(f'Средняя прибыль у фильмов с высоким бюджетом: {round(g2.profit_or_loss.mean(), 2)}')
print()

print('Проверка нормальности распределения прибыли')
print(ss.shapiro(g1.profit_or_loss))
print(ss.shapiro(g2.profit_or_loss))
print()

print('Проверка на равенство дисперсий')
print(ss.levene(g1.profit_or_loss, g2.profit_or_loss))
print()

Средняя прибыль у фильмов с низким бюджетом: 21334045.83
Средняя прибыль у фильмов с высоким бюджетом: 173973770.39

Проверка нормальности распределения прибыли
ShapiroResult(statistic=np.float64(0.522051163613692), pvalue=np.float64(1.6064660509650426e-40))
ShapiroResult(statistic=np.float64(0.6649482551300938), pvalue=np.float64(2.664180681822979e-35))

Проверка на равенство дисперсий
LeveneResult(statistic=np.float64(208.7560024319817), pvalue=np.float64(2.5882162738827526e-44))



Полученные результаты тестов свидетельствуют о том, что данные распределены ненормально, а дисперсии в рассматриваемых выборках неравны. Вследствие этого использование классического t-теста для определения уровня статистической значимости разниц становится  некорректным: вместо указанного теста необходимо использовать непараметрический тест Манна-Уитни:

In [16]:
ss.mannwhitneyu(g1.profit_or_loss, g2.profit_or_loss)

MannwhitneyuResult(statistic=np.float64(133394.0), pvalue=np.float64(2.974320257075564e-60))

**Вывод : Результаты непараметрического теста Манна-Уитни свидетельствуют о наличии статистически значимой разницы между выборками. Таким образом, гипотеза о том, что фильмы с более высоким бюджетом имеют большую прибыль, подтверждается — в среднем такие фильмы действительно приносят значительно больше прибыли.**